# ZoteroRetriever

这将帮助你开始使用 Zotero [retriever](/docs/concepts/retrievers)。有关 ZoteroRetriever 所有功能和配置的详细文档，请访问 [Github 页面](https://github.com/TimBMK/langchain-zotero-retriever)。

### 集成详情


| Retriever | Source | Package |
| :--- | :--- | :---: |
[ZoteroRetriever](https://github.com/TimBMK/langchain-zotero-retriever) | [Zotero API](https://www.zotero.org/support/dev/web_api/v3/start) | langchain-community |

## 设置

如果你想从单个查询中获得自动跟踪，你也可以通过取消注释下方内容来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

这个检索器位于 `langchain-zotero-retriever` 包中。我们也需要 `pyzotero` 依赖：

In [ ]:
%pip install -qU langchain-zotero-retriever pyzotero

## 实例化

`ZoteroRetriever` 的参数包括：
- `k`: 要包含的结果数量 (默认值: 50)
- `type`: 要执行的搜索类型。“Top”检索顶层 Zotero 库项目，“items”返回任何 Zotero 库项目。(默认值: top)
- `get_fulltext`: 如果项目附加有全文，则检索全文。如果为 False 或未附加文本，则返回空字符串作为 page_content。(默认值: True)
- `library_id`: 要搜索的 Zotero 库的 ID。连接到库是必需的。
- `library_type`: 要搜索的库类型。“user”代表个人库，“group”代表共享群组库。(默认值: user)
- `api_key`: 如果未设置为环境变量，则为 Zotero API 密钥。可选，访问非公开群组库或个人库需要此项。如果提供了 ZOTERO_API_KEY 环境变量，则自动获取。

In [ ]:
from langchain_zotero_retriever.retrievers import ZoteroRetriever

retriever = ZoteroRetriever(
    k=10,
    library_id="2319375",  # a public group library that does not require an API key for access
    library_type="group",  # set this to "user" if you are using a personal library. Personal libraries require an API key
)

## 用法

除了 `query` 之外，检索器还提供以下附加搜索参数：
- `itemType`: 要搜索的条目类型（例如，“book”或“journalArticle”）
- `tag`: 用于搜索附加到库条目的标签（请参阅搜索语法以组合多个标签）
- `qmode`: 要使用的搜索模式。更改查询搜索的内容。“everything”包括全文内容。“titleCreatorYear”用于搜索标题、作者和年份。
- `since`: 仅返回在指定库版本之后修改过的对象。默认为返回所有对象。

有关搜索语法，请参阅 Zotero API 文档：https://www.zotero.org/support/dev/web_api/v3/basics#search_syntax

有关完整的 API 架构（包括可用的 itemTypes），请参阅：https://github.com/zotero/zotero-schema

In [ ]:
query = "Zuboff"

retriever.invoke(query)

In [ ]:
tags = [
    "Surveillance",
    "Digital Capitalism",
]  # note that providing tags as a list will result in a logical AND operation

retriever.invoke("", tag=tags)

## 在链中使用

由于 Zotero API 的搜索方式，直接将用户问题传递给 ZoteroRetriever 通常无法获得令人满意的结果。在链或代理框架中使用时，建议将 ZoteroRetriever 转换为一个[工具](https://python.langchain.com/docs/how_to/custom_tools/#creating-tools-from-functions)。这样，LLM 就可以将用户查询转换为更简洁的 API 搜索查询。此外，这还允许 LLM 填充其他搜索参数，例如标签或项目类型。

In [ ]:
from typing import List, Optional, Union

from langchain_core.output_parsers import PydanticToolsParser
from langchain_core.tools import StructuredTool, tool
from langchain_openai import ChatOpenAI


def retrieve(
    query: str,
    itemType: Optional[str],
    tag: Optional[Union[str, List[str]]],
    qmode: str = "everything",
    since: Optional[int] = None,
):
    retrieved_docs = retriever.invoke(
        query, itemType=itemType, tag=tag, qmode=qmode, since=since
    )
    serialized_docs = "\n\n".join(
        (
            f"Metadata: { {key: doc.metadata[key] for key in doc.metadata if key != 'abstractNote'} }\n"
            f"Abstract: {doc.metadata['abstractNote']}\n"
        )
        for doc in retrieved_docs
    )

    return serialized_docs, retrieved_docs


description = """Search and return relevant documents from a Zotero library. The following search parameters can be used:

    Args:
        query: str: The search query to be used. Try to keep this specific and short, e.g. a specific topic or author name
        itemType: Optional. Type of item to search for (e.g. "book" or "journalArticle"). Multiple types can be passed as a string separated by "||", e.g. "book || journalArticle". Defaults to all types.
        tag: Optional. For searching over tags attached to library items. If documents tagged with multiple tags are to be retrieved, pass them as a list. If documents with any of the tags are to be retrieved, pass them as a string separated by "||", e.g. "tag1 || tag2"
        qmode: Search mode to use. Changes what the query searches over. "everything" includes full-text content. "titleCreatorYear" to search over title, authors and year. Defaults to "everything".
        since: Return only objects modified after the specified library version. Defaults to return everything.
    """

retriever_tool = StructuredTool.from_function(
    func=retrieve,
    name="retrieve",
    description=description,
    return_direct=True,
)


llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

llm_with_tools = llm.bind_tools([retrieve])

q = "What journal articles do I have on Surveillance in the zotero library?"

chain = llm_with_tools | PydanticToolsParser(tools=[retrieve])

chain.invoke(q)

## API 参考

有关所有 ZoteroRetriever 功能和配置的详细文档，请访问 [Github 页面](https://github.com/TimBMK/langchain-zotero-retriever)。

有关 Zotero API 的详细文档，请访问 [Zotero API 参考](https://www.zotero.org/support/dev/web_api/v3/start)。